In [ ]:
# 回测（高级 API）

[NautilusTrader](https://nautilustrader.io/docs/) 高性能算法交易平台和事件驱动回测器教程。

[在 GitHub 上查看源码](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/getting_started/backtest_high_level.ipynb)。


In [ ]:
## 概述

本教程将介绍如何使用 `BacktestNode` 在模拟 FX ECN 场所上使用历史报价 tick 数据回测简单的 EMA 交叉策略。

将涵盖以下要点：
- 如何将原始数据（Nautilus 外部）加载到数据目录中
- 如何为 `BacktestNode` 设置配置对象
- 如何使用 `BacktestNode` 运行回测


In [ ]:
## 先决条件
- 已安装 Python 3.11+
- 已安装 [JupyterLab](https://jupyter.org/) 或类似软件 (`pip install -U jupyterlab`)
- 已安装 [NautilusTrader](https://pypi.org/project/nautilus_trader/) 最新版本 (`pip install -U nautilus_trader`)


In [ ]:
## 导入

我们将从本教程其余部分的所有导入开始。


In [ ]:
import shutil
from decimal import Decimal
from pathlib import Path

import pandas as pd

from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import CSVTickDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider


In [ ]:
作为开始笔记本之前的一次性操作 - 我们需要下载一些用于回测的示例数据。

对于这个例子，我们将使用来自 `histdata.com` 的 FX 数据。只需访问 https://www.histdata.com/download-free-forex-historical-data/?/ascii/tick-data-quotes/ 并选择一个 FX 货币对，然后选择一个或多个月的数据进行下载。

下载文件的示例：

* `DAT_ASCII_EURUSD_T_202410.csv` (2024-10 月的 EUR\USD 数据)
* `DAT_ASCII_EURUSD_T_202411.csv` (2024-11 月的 EUR\USD 数据)

下载数据后：

1. 将上述文件复制到一个文件夹中 - 例如：`~/Downloads/Data/`（默认情况下，它将使用用户的 `Downloads/Data/` 目录）
2. 将下面的变量 `DATA_DIR` 设置为包含数据的目录。


In [ ]:
DATA_DIR = "~/Downloads/Data/"


In [ ]:
path = Path(DATA_DIR).expanduser()
raw_files = list(path.iterdir())
assert raw_files, f"Unable to find any histdata files in directory {path}"
raw_files


In [ ]:
## 将数据加载到 Parquet 数据目录

来自 `histdata` 的 FX 数据以 CSV/文本格式存储，字段为 `timestamp, bid_price, ask_price`。
首先，我们需要将这些原始数据加载到具有与 Nautilus 报价兼容架构的 `pandas.DataFrame` 中。

然后我们可以通过使用 `QuoteTickDataWrangler` 处理 DataFrame 来创建 Nautilus `QuoteTick` 对象。


In [ ]:
# 这里我们只是取找到的第一个数据文件并加载到 pandas DataFrame
df = CSVTickDataLoader.load(
    file_path=raw_files[0],                                   # 输入第 1 个 CSV 文件
    index_col=0,                                              # 使用数据中的第 1 列作为数据框的索引
    header=None,                                              # CSV 文件中没有列名
    names=["timestamp", "bid_price", "ask_price", "volume"],  # 指定各个列的名称
    usecols=["timestamp", "bid_price", "ask_price"],          # 只从 CSV 文件读取这些列到数据框
    parse_dates=["timestamp"],                                # 指定包含日期/时间的列
    date_format="%Y%m%d %H%M%S%f",                            # 解析日期时间的格式
)

# 让我们确保数据按时间戳排序
df = df.sort_index()

# 加载的数据框预览
df.head(2)


In [ ]:
# 使用整理器处理报价
EURUSD = TestInstrumentProvider.default_fx_ccy("EUR/USD")
wrangler = QuoteTickDataWrangler(EURUSD)

ticks = wrangler.process(df)

# 预览：查看前 2 个 tick
ticks[0:2]


In [ ]:
有关更多详细信息，请参阅[加载数据](../concepts/data)指南。

接下来，我们只需实例化一个 `ParquetDataCatalog`（传入存储数据的目录，默认情况下我们将只使用当前目录）。
然后我们可以将工具和 tick 数据写入目录，加载数据应该只需要几分钟（取决于有多少个月）。


In [ ]:
CATALOG_PATH = Path.cwd() / "catalog"

# 如果已存在则清除，然后创建新的
if CATALOG_PATH.exists():
    shutil.rmtree(CATALOG_PATH)
CATALOG_PATH.mkdir(parents=True)

# 创建目录实例
catalog = ParquetDataCatalog(CATALOG_PATH)

# 将工具写入目录
catalog.write_data([EURUSD])

# 将 tick 写入目录
catalog.write_data(ticks)


In [ ]:
## 使用数据目录

一旦数据被加载到目录中，`catalog` 实例就可以用于为回测加载数据，或者仅用于研究目的。
它包含从目录中提取数据的各种方法，例如 `.instruments(...)` 和 `quote_ticks(...)`（如下所示）。


In [ ]:
# 获取目录中所有工具的列表
catalog.instruments()


In [ ]:
# 从目录中查看第 1 个工具
instrument = catalog.instruments()[0]
instrument


In [ ]:
# 从目录查询报价 tick
start = dt_to_unix_nanos(pd.Timestamp("2024-10-01", tz="UTC"))
end =  dt_to_unix_nanos(pd.Timestamp("2024-10-15", tz="UTC"))
selected_quote_ticks = catalog.quote_ticks(instrument_ids=[EURUSD.id.value], start=start, end=end)

# 预览第一个
selected_quote_ticks[:2]


In [ ]:
## 添加场所


In [ ]:
venue_configs = [
    BacktestVenueConfig(
        name="SIM",
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["1_000_000 USD"],
    ),
]


In [ ]:
## 添加数据


In [ ]:
str(CATALOG_PATH)


In [ ]:
data_configs = [
    BacktestDataConfig(
        catalog_path=str(CATALOG_PATH),
        data_cls=QuoteTick,
        instrument_id=instrument.id,
        start_time=start,
        end_time=end,
    ),
]


In [ ]:
## 添加策略


In [ ]:
strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config={
            "instrument_id": instrument.id,
            "bar_type": "EUR/USD.SIM-15-MINUTE-BID-INTERNAL",
            "fast_ema_period": 10,
            "slow_ema_period": 20,
            "trade_size": Decimal(1_000_000),
        },
    ),
]


In [ ]:
## 配置回测

Nautilus 使用 `BacktestRunConfig` 对象，它可以在一个地方配置回测。它是一个 `Partialable` 对象（这意味着它可以分阶段配置）；其好处是在创建多个回测运行时减少样板代码（例如，在对参数进行某种网格搜索时）。


In [ ]:
config = BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)


In [ ]:
## 运行回测

现在我们可以运行回测节点，它将在整个数据流上模拟交易。


In [ ]:
node = BacktestNode(configs=[config])

results = node.run()
results
